In [238]:
import pandas as pd
import random
import numpy as np

In [251]:
df = pd.read_csv('/kaggle/input/enron-dataset/enron.csv')
rows_to_remove = df[df['Unnamed: 0'].str.contains('TOTAL')].index
df = df.drop(rows_to_remove)
df = df.drop(['Unnamed: 0','email_address'], axis=1)

poi = df[df.loc[:,'poi']==True]#.fillna(0)
not_poi = pd.DataFrame(df[df.loc[:,'poi']==False])#.fillna(0)

In [252]:
def make_new_records(df, records_num):
    new_df = pd.DataFrame()
    for i in range(records_num):
        sample_times = random.randint(2,4)
        sample_idx = random.sample(range(0,len(df.index)), sample_times)
        sampled_df = df.iloc[[idx for idx in sample_idx]]
        numeric_cols = sampled_df.select_dtypes(include=np.number)
        avg_row = numeric_cols.mean()
        avg_row['poi'] = sampled_df.loc[sampled_df.index[0],'poi']
        new_df = pd.concat([new_df, avg_row.to_frame().T], ignore_index=True)
    return new_df
def create_new_dataset(poi_df, not_poi_df, poi_num=20, not_poi_num=40):
    new_poi = make_new_records(poi_df, poi_num)
    new_not_poi = make_new_records(not_poi_df, not_poi_num)
    return pd.concat([new_poi, new_not_poi], axis=0, ignore_index=True)

In [253]:
dataset = create_new_dataset(poi, not_poi, 60, 450)
dataset

,salary,to_messages,deferral_payments,total_payments,loan_advances,bonus,restricted_stock_deferred,deferred_income,total_stock_value,expenses,from_poi_to_this_person,exercised_stock_options,from_messages,other,from_this_person_to_poi,long_term_incentive,shared_receipt_with_poi,restricted_stock,director_fees,poi
0,351865.5,1825.0,18934.5,1150065.0,NaN,1100000.0,NaN,-189531.0,11322123.0,41722.0,149.0,30766064.0,44.5,137497.0,11.5,350000.0,1358.5,1600152.5,NaN,True
1,229215.0,1505.333333,NaN,890140.75,NaN,1116666.666667,NaN,-1689755.5,4795099.0,62445.5,94.333333,4743760.333333,64.0,90023.5,17.333333,687779.333333,1365.666667,1237278.75,NaN,True
2,275803.75,1035.333333,214678.0,809183.5,NaN,1016666.666667,NaN,-1802193.0,9281182.5,65036.5,43.0,7530673.25,38.666667,131261.75,19.333333,844017.0,998.666667,1750509.25,NaN,True
3,274110.666667,975.0,NaN,384677.333333,NaN,1625000.0,NaN,-3310698.5,13606120.666667,44978.666667,37.0,11213576.0,25.0,111379.666667,12.5,1617011.0,949.5,2392544.666667,NaN,True
4,271260.333333,2245.0,NaN,1289451.666667,NaN,1125000.0,NaN,-248750.0,4837616.0,40146.333333,128.5,8308552.0,96.5,152110.333333,18.5,362652.0,1886.5,2068098.666667,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,176727.333333,496.0,NaN,1989793.666667,NaN,450000.0,NaN,-280000.0,2499017.666667,32660.0,28.0,984703.333333,19.0,674648.333333,5.0,899091.333333,437.0,1514314.333333,NaN,False
506,316594.0,2202.5,77716.0,2180129.5,NaN,1375000.0,NaN,NaN,2339399.0,121385.0,57.0,1539466.0,1366.5,1081.5,49.0,327211.0,1293.0,799933.0,NaN,False
507,222454.0,NaN,NaN,1994892.5,NaN,450000.0,NaN,-280000.0,838536.5,34943.5,NaN,394469.0,NaN,235108.0,NaN,1192387.0,NaN,444067.5,NaN,False
508,263908.5,1916.5,94977.0,778107.75,NaN,600000.0,NaN,-121284.0,1538357.333333,81364.0,54.0,1492692.5,1377.0,351.0,97.0,554422.0,1119.5,814843.5,125034.0,False


In [255]:
dataset.to_csv('/kaggle/working/new_records.csv')